In [17]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import pickle
from collections import Counter

In [18]:
from sklearn.cluster import DBSCAN, SpectralClustering, MeanShift,KMeans
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn.cluster import AgglomerativeClustering

In [19]:
df = pd.read_csv('reclustering.csv')
pd.set_option("display.max_columns", 101)

In [20]:
df.sample(10)

,_Accel 10,Accel 25,Accel 50,Accel 75,Accel 90,Accel Ave,bearingRate 10,bearingRate 25,bearingRate 50,bearingRate 75,bearingRate 90,bearingRate ave,Cluster,Distance Traveled,F1,Jerk 10,Jerk 25,Jerk 50,Jerk 75,Jerk 90,Jerk Ave,Last Lat,Last Long,Number of Records,Speed 10,Speed 25,Speed 50,Speed 75,Speed 90,Speed Ave,Start Lat,Start Long,Time Traveled
2138,-1.263370,-0.332402,0.074001,0.789001,1.175721,0.056973,6.720000e-06,0.000013,0.000020,0.000066,0.000072,0.000034,1,0.073424,0,-0.649908,-0.218637,0.043533,0.249735,0.489442,-0.010016,39.987185,116.303363,1,1.351152,2.594477,3.326402,4.152303,6.787324,3.622853,39.986612,116.302657,1.150000
4124,-2.126872,-0.932400,-0.039601,0.806399,2.188790,0.034057,1.440000e-12,0.000002,0.000004,0.000009,0.000018,0.000009,1,1.386111,0,-3.606448,-1.404068,0.215952,1.731676,4.116246,-0.017621,39.974937,116.329938,1,1.566000,2.095196,2.894394,3.873589,5.749188,3.508200,39.975123,116.311028,23.216667
3948,-0.470303,-0.201815,0.004609,0.208656,0.552040,0.021784,5.370000e-06,0.000012,0.000031,0.000069,0.000124,0.000060,1,1.131653,0,-0.196614,-0.060897,0.011375,0.086905,0.174978,-0.002668,40.010800,116.321913,1,0.626399,1.594081,2.908799,3.779278,5.718813,3.213798,39.999873,116.326869,20.216667
1572,-1.109520,-0.464572,0.049277,0.591072,1.285160,0.100932,4.570000e-06,0.000014,0.000045,0.000134,0.000297,0.000113,0,14.938302,0,-0.356408,-0.154864,0.015776,0.155840,0.357160,-0.007433,39.975587,116.331147,1,1.693259,5.555244,10.908302,21.891906,31.418959,14.748463,40.075935,116.323208,47.433333
4128,-1.171608,-0.560003,-0.052400,0.459201,1.048300,-0.040204,4.430000e-06,0.000009,0.000018,0.000034,0.000056,0.000027,1,3.227041,0,-0.625097,-0.255249,-0.003732,0.245466,0.567907,0.005252,39.984983,116.353543,1,1.946882,3.082802,5.540388,16.750833,23.240206,9.938217,39.945818,116.350147,23.283333
1341,-0.806461,-0.337387,-0.010793,0.219606,0.628205,0.015111,2.150000e-06,0.000004,0.000013,0.000038,0.000110,0.000114,0,9.643323,0,-0.469357,-0.131718,0.012148,0.189284,0.446843,0.003458,39.975220,116.330925,1,1.326597,2.322000,3.142792,6.273893,40.378224,10.059180,40.075327,116.327705,37.750000
4602,-0.043776,-0.017763,0.003470,0.016766,0.024654,-0.006662,5.590000e-05,0.000157,0.000313,0.001007,0.003190,0.000991,1,2.454278,0,-0.001130,-0.000453,-0.000059,0.000295,0.001211,-0.000135,39.991567,116.332717,1,1.726722,4.051787,5.977878,6.722451,7.200449,5.182494,39.975383,116.327967,35.016667
3789,-0.972011,-0.419407,0.000000,0.417596,1.098904,0.019095,2.150000e-06,0.000004,0.000011,0.000020,0.000038,0.000022,1,1.413864,0,-0.875692,-0.302844,0.008325,0.343338,0.859944,-0.002393,39.979708,116.313043,1,1.672196,2.244595,3.169793,4.543209,6.877814,3.997760,39.975242,116.328995,17.750000
4664,-1.460250,-0.748911,0.031500,0.797287,1.669187,0.041164,4.430000e-06,0.000011,0.000034,0.000101,0.000182,0.000075,1,2.764464,0,-0.682264,-0.280171,0.018703,0.267538,0.630855,-0.018334,39.979487,116.308723,1,0.977062,3.093303,8.595901,15.603593,22.331161,10.463069,39.975328,116.329138,36.800000
2333,-0.120801,-0.087364,-0.019447,0.093138,0.151396,0.008581,9.400000e-05,0.000157,0.000269,0.000649,0.000886,0.000444,1,0.597508,0,-0.015202,-0.007764,-0.002803,0.006440,0.010485,-0.002059,39.975033,116.328917,1,5.984560,6.984829,9.247567,10.986731,12.628379,9.040301,39.967400,116.328217,3.466667


In [12]:
df_cluster = df[['Last Lat', 'Last Long']]

In [21]:
X = StandardScaler()
X = X.fit_transform(df_cluster)

In [29]:
scores = []
EP =[]
number =[]
eps= [.08,.10,.15,.20,.25,.30,.40,.50,.60]
size= [200,250,300]
for ep in eps:
    for siz in size:
        db = DBSCAN(eps=ep, min_samples=siz)   
        pred = db.fit_predict(X)
        
        print('ep:'+ str(ep) + ' size:'+ str(siz))
        print(Counter(pred))
        try:
            scores.append(silhouette_score(X, pred))
        except:
            scores.append(np.nan)
        EP.append(ep)
        number.append(siz)



ep:0.08 size:200
Counter({-1: 3721, 0: 2030, 1: 469, 2: 443})
ep:0.08 size:250
Counter({-1: 3928, 0: 1843, 1: 469, 2: 423})
ep:0.08 size:300
Counter({-1: 3997, 0: 1788, 1: 465, 2: 413})
ep:0.1 size:200
Counter({-1: 3298, 0: 2138, 2: 494, 1: 485, 3: 248})
ep:0.1 size:250
Counter({-1: 3572, 0: 2122, 2: 485, 1: 484})
ep:0.1 size:300
Counter({-1: 3704, 0: 2007, 1: 481, 2: 471})
ep:0.15 size:200
Counter({-1: 2699, 0: 2313, 1: 590, 2: 546, 3: 332, 4: 183})
ep:0.15 size:250
Counter({-1: 2918, 0: 2301, 1: 590, 2: 534, 3: 320})
ep:0.15 size:300
Counter({-1: 3260, 0: 2293, 1: 584, 2: 526})
ep:0.2 size:200
Counter({0: 2584, -1: 2297, 1: 664, 2: 602, 3: 516})
ep:0.2 size:250
Counter({0: 2536, -1: 2368, 1: 650, 2: 598, 3: 511})
ep:0.2 size:300
Counter({0: 2514, -1: 2414, 1: 649, 2: 587, 3: 499})
ep:0.25 size:200
Counter({0: 2765, -1: 1836, 1: 1453, 2: 609})
ep:0.25 size:250
Counter({0: 2736, -1: 1942, 1: 760, 2: 685, 3: 540})
ep:0.25 size:300
Counter({0: 2654, -1: 2033, 1: 756, 2: 683, 3: 537})
ep:

In [53]:
db = DBSCAN(eps= 0.37, min_samples = 250)
pred = db.fit_predict(X)

In [54]:
df['final_location_cluster'] = pred

In [55]:
df.to_csv('with_final_location.csv')

In [ ]:
df_cluster = df[['Start Lat', 'Start Long']]